LetsGrowMore Virtual Internship Program

Task : Next Word Prediction.

 Import the required libraries

In [2]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

Now load the dataset

In [3]:
text = open('1661-0.txt').read().lower()
print('corpus length:', len(text))

corpus length: 581888


Now, we want to split the entire dataset into each word in order without the presence of special characters.

In [4]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

Next, for the feature engineering part, we need to have the unique sorted words list. We also need a dictionary with each word form the unique_words list as key and its corresponding position as value.

In [5]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

Feature engineering

In [6]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


Here, we create two numpy array X(for storing the features) and Y(for storing the corresponding label).

In [7]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

In [8]:
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [9]:
print(X[0][0])

[False False False ... False False False]


Building the model

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

Training

In [12]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/20
811/811 [==============================] - 290s 353ms/step - loss: 6.0101 - accuracy: 0.1073 - val_loss: 7.0723 - val_accuracy: 0.1033
Epoch 2/20
811/811 [==============================] - 285s 352ms/step - loss: 5.7724 - accuracy: 0.1482 - val_loss: 7.7905 - val_accuracy: 0.1023
Epoch 3/20
811/811 [==============================] - 287s 354ms/step - loss: 5.7672 - accuracy: 0.1746 - val_loss: 7.9921 - val_accuracy: 0.1100
Epoch 4/20
811/811 [==============================] - 285s 352ms/step - loss: 5.4483 - accuracy: 0.2089 - val_loss: 8.2435 - val_accuracy: 0.1077
Epoch 5/20
811/811 [==============================] - 283s 349ms/step - loss: 5.1461 - accuracy: 0.2487 - val_loss: 8.2791 - val_accuracy: 0.0932
Epoch 6/20
811/811 [==============================] - 281s 346ms/step - loss: 4.8995 - accuracy: 0.2872 - val_loss: 8.5497 - val_accuracy: 0.0921
Epoch 7/20
811/811 [==============================] - 283s 349ms/step - loss: 4.6633 - accuracy: 0.3270 - val_loss: 8.7364 -

After successful training, we will save the trained model and just load it back as needed.

In [13]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))

model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

Prediction

In [14]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

To choose the best possible n words after the prediction from the model is done by sample function.

In [15]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

Finally, for prediction, we use the function predict_completions which use the model to predict and return the list of n predicted words.

In [16]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

Now let’s see how it predicts, we use tokenizer.tokenize fo removing the punctuations and also we choose 5 first words because our predicts base on 5 previous words.

In [17]:
q =  "Your life will never be there in the same situation again"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Your life will never be there in the same situation again
Sequence:  your life will never be
your
life
will
never
be
next possible words:  ['and', 'at', 'that', 'very', 'too']
